### Before the start

In [1]:
# # before start
# %pip install tld
# %pip install category_encoders

# ! pip install pytz
# !pip install timezonefinder
# !pip install geopy

# !pip3 install torch
# !pip3 install torchvision
# !pip3 install tqdm

# !pip install torch -q
# !pip install keras

# !pip install skorch
# !pip install bayesian-optimization

# # !pip install cudf
# # !pip install cudf-cu11==22.12 rmm-cu11==22.12 dask-cudf-cu11==22.12 --extra-index-url=https://pypi.ngc.nvidia.com/

In [2]:
# load data and import libraries
import pandas as pd
import numpy as np
import pytz
# from geopy.geocoders import Nominatim
# from timezonefinder import TimezoneFinder
import datetime

import torch
# For layers and models
import torch.nn as nn
# For activation functions (F.sigmoid is deprecated)
import torch.nn.functional as F
from torch import sigmoid

import os
import warnings
warnings.filterwarnings("ignore")
from torch.utils.data import Dataset, DataLoader
# Collection of optimizers
import torch.optim as optim

# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
# model = CustomModel().to(device)

### ANN preprocessing

In [3]:
new_labels = [
    "Sport",
    "Hobby Passive",
    "Work&Study",
    "Daily Routine"
]
match_labels_v3 = [0,3,0,1,2,2,3]

match_labels = {
    "Sport": 0,
    "Food": 3,
    "Hobby Active": 0,
    "Hobby Passive": 1,
    "Studying":2,
    "Work":2,
    "Other":3
}


In [4]:
df = pd.read_csv('schedule.csv')

df['Label Number'] = df['Label'].apply(lambda x: match_labels[x])
df['Label Name'] = df['Label Number'].apply(lambda x: new_labels[x])

df.drop(columns=['Label', 'Activity', 'End Time', 'If Has Deadline', 'Deadline Time', 'Deadline Date'], inplace=True)

mini = df['Importance'].min()
maxi = df['Importance'].max()
df['Importance'] = df['Importance'].apply(lambda x: int((x-mini) / (maxi-mini) * 3) )

df_out = df[208:]

df[:13]

,Date,Start Time,Duration,Importance,Label Number,Label Name
0,2023-06-16,07:00,60,1,3,Daily Routine
1,2023-06-16,08:15,45,2,0,Sport
2,2023-06-16,09:15,60,0,1,Hobby Passive
3,2023-06-16,10:30,45,3,2,Work&Study
4,2023-06-16,11:30,60,1,3,Daily Routine
5,2023-06-16,13:00,60,2,2,Work&Study
6,2023-06-16,14:15,30,0,0,Sport
7,2023-06-16,15:00,90,1,2,Work&Study
8,2023-06-16,17:00,45,1,3,Daily Routine
9,2023-06-16,18:00,45,2,0,Sport


In [5]:
from sklearn.preprocessing import OneHotEncoder

def encode_features(df, features_name, encoder):
    new_features = encoder.transform(df[features_name])
    # create dataframe from encoded features with named columns
    new_columns = pd.DataFrame(new_features, columns=encoder.get_feature_names_out())
    new_df = pd.concat([df, new_columns], axis=1)
    # new_df.drop(features_name, axis=1, inplace=True)
    return new_df

encoder = OneHotEncoder(sparse=False, drop='first')
enc_column = ['Label Number']
encoder.fit(df[enc_column])
df = encode_features(df, enc_column, encoder)

df

,Date,Start Time,Duration,Importance,Label Number,Label Name,Label Number_1,Label Number_2,Label Number_3
0,2023-06-16,07:00,60,1,3,Daily Routine,0.0,0.0,1.0
1,2023-06-16,08:15,45,2,0,Sport,0.0,0.0,0.0
2,2023-06-16,09:15,60,0,1,Hobby Passive,1.0,0.0,0.0
3,2023-06-16,10:30,45,3,2,Work&Study,0.0,1.0,0.0
4,2023-06-16,11:30,60,1,3,Daily Routine,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
249,2023-07-07,15:45,90,2,2,Work&Study,0.0,1.0,0.0
250,2023-07-07,17:30,45,1,3,Daily Routine,0.0,0.0,1.0
251,2023-07-07,18:30,90,1,0,Sport,0.0,0.0,0.0
252,2023-07-07,20:15,90,2,3,Daily Routine,0.0,0.0,1.0


In [7]:
def make_true_duration(x):
    true_noise = [20, -15, 20, -10]
    result= x['Duration'] + true_noise[x['Label Number']] + int(np.random.uniform(low=-5, high=5))

    if result < 10:
        return 10

    return result

df['True Duration'] = df.apply(lambda x: make_true_duration(x), axis=1)
df

,Date,Start Time,Duration,Importance,Label Number,Label Name,Label Number_1,Label Number_2,Label Number_3,True Duration
0,2023-06-16,07:00,60,1,3,Daily Routine,0.0,0.0,1.0,50
1,2023-06-16,08:15,45,2,0,Sport,0.0,0.0,0.0,67
2,2023-06-16,09:15,60,0,1,Hobby Passive,1.0,0.0,0.0,41
3,2023-06-16,10:30,45,3,2,Work&Study,0.0,1.0,0.0,63
4,2023-06-16,11:30,60,1,3,Daily Routine,0.0,0.0,1.0,50
...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,15:45,90,2,2,Work&Study,0.0,1.0,0.0,107
250,2023-07-07,17:30,45,1,3,Daily Routine,0.0,0.0,1.0,32
251,2023-07-07,18:30,90,1,0,Sport,0.0,0.0,0.0,113
252,2023-07-07,20:15,90,2,3,Daily Routine,0.0,0.0,1.0,82


In [9]:
# def make_true_duration(x, prob=1.3):
#     true_noise = [[-5,20], [-15,0], [-20,5], [-10,30]]
#     addi = np.random.uniform(low=prob*true_noise[x['Label Number']][0], high=prob * true_noise[x['Label Number']][1])
#     result= x['Duration'] + int(addi)
#     if addi > true_noise[x['Label Number']][1]:
#         result= x['Duration'] + true_noise[x['Label Number']][1]
#     elif addi < true_noise[x['Label Number']][0]:
#         result= x['Duration'] + true_noise[x['Label Number']][0]
#     if result < 10:
#         return 10
#     return result

# df['True Duration'] = df.apply(lambda x: make_true_duration(x), axis=1)
# df

In [10]:
min_time = datetime.datetime.strptime("00:00", '%H:%M')
max_time = ((datetime.datetime.strptime("23:59", '%H:%M')-min_time).total_seconds()+1)//60
# df['Start Time'] = df['Start Time'].apply(lambda x: ((datetime.datetime.strptime(x, '%H:%M')-min_time).total_seconds()+1)/max_time)
df['Start Time'] = df['Start Time'].apply(lambda x: (datetime.datetime.strptime(x, '%H:%M')-min_time).total_seconds()//60)

# move start time
cur_day = '1/1/2020'
delta = 0

for index, row in df.iterrows():
    if row['Date'] != cur_day:
        delta = row['Start Time']
        cur_day = row['Date']
    df.at[index, 'Start Time'] -= delta

df['True Start Time'] = df['Start Time']

df_out['True Duration'] = df['True Duration']
df_out['True Hours'] = df['True Start Time']//24
df_out['True Minutes'] = df['True Start Time']%24

df.describe()

,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time
count,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000
mean,426.496063,66.259843,1.251969,1.779528,0.110236,0.220472,0.409449,70.070866,426.496063
std,278.493487,19.999689,0.897678,1.231429,0.313802,0.415384,0.492703,30.327248,278.493487
min,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000
25%,180.000000,45.000000,1.000000,0.000000,0.000000,0.000000,0.000000,47.000000,180.000000
50%,420.000000,60.000000,1.000000,2.000000,0.000000,0.000000,0.000000,63.000000,420.000000
75%,660.000000,90.000000,2.000000,3.000000,0.000000,0.000000,1.000000,107.750000,660.000000
max,900.000000,90.000000,3.000000,3.000000,1.000000,1.000000,1.000000,114.000000,900.000000


In [11]:
TS_DUR =10
LENGTH = 1200

def parse_date(d):
    res = [int(i) for i in d.split('/')[::-1]]
    return datetime.date(*res)

cur_day = '1/1/2020'
# basic_slots = [0]*(2*60//10)+[1]*(3*60//10)+[0]*(2*60//10)
basic_slots = [0]*(LENGTH//TS_DUR)

available_time_slots = [0]*(LENGTH//TS_DUR)
names = ['TS'+str(i) for i in range(len(available_time_slots))]
df[names] = available_time_slots

for index, row in df.iterrows():
    if row['Date'] == cur_day:
        df.loc[index, names] = available_time_slots.copy()
    else:
        cur_day = row['Date']
        available_time_slots = basic_slots.copy()
        df.loc[index, names] = available_time_slots.copy()
    start = int(row['True Start Time'] / TS_DUR)
    end = int((row['True Start Time'] + row['True Duration']) // TS_DUR) + [1 if (row['True Start Time'] + row['True Duration']) % TS_DUR > 0 else 0][0]
    # print(start, end, row['True Duration'] )
    df.loc[index, 'Summ'] = sum(available_time_slots)

    available_time_slots[start:end+1] = [1]*(end-start+1)
    # print(index, available_time_slots)

df

,Date,Start Time,Duration,Importance,Label Number,Label Name,Label Number_1,Label Number_2,Label Number_3,True Duration,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
0,2023-06-16,0.0,60,1,3,Daily Routine,0.0,0.0,1.0,50,...,0,0,0,0,0,0,0,0,0,0.0
1,2023-06-16,75.0,45,2,0,Sport,0.0,0.0,0.0,67,...,0,0,0,0,0,0,0,0,0,6.0
2,2023-06-16,135.0,60,0,1,Hobby Passive,1.0,0.0,0.0,41,...,0,0,0,0,0,0,0,0,0,15.0
3,2023-06-16,210.0,45,3,2,Work&Study,0.0,1.0,0.0,63,...,0,0,0,0,0,0,0,0,0,18.0
4,2023-06-16,270.0,60,1,3,Daily Routine,0.0,0.0,1.0,50,...,0,0,0,0,0,0,0,0,0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,525.0,90,2,2,Work&Study,0.0,1.0,0.0,107,...,0,0,0,0,0,0,0,0,0,52.0
250,2023-07-07,630.0,45,1,3,Daily Routine,0.0,0.0,1.0,32,...,0,0,0,0,0,0,0,0,0,62.0
251,2023-07-07,690.0,90,1,0,Sport,0.0,0.0,0.0,113,...,0,0,0,0,0,0,0,0,0,65.0
252,2023-07-07,795.0,90,2,3,Daily Routine,0.0,0.0,1.0,82,...,0,0,0,0,0,0,0,0,0,78.0


In [12]:
# scaling
DUR = 6*60
df['Importance'] = df['Importance'] / 3

df.drop(columns=['Label Name'], inplace=True)

df['Duration'] = df['Duration'] / DUR
df['True Duration'] = df['True Duration'] / DUR

df

,Date,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
0,2023-06-16,0.0,0.166667,0.333333,3,0.0,0.0,1.0,0.138889,0.0,...,0,0,0,0,0,0,0,0,0,0.0
1,2023-06-16,75.0,0.125000,0.666667,0,0.0,0.0,0.0,0.186111,75.0,...,0,0,0,0,0,0,0,0,0,6.0
2,2023-06-16,135.0,0.166667,0.000000,1,1.0,0.0,0.0,0.113889,135.0,...,0,0,0,0,0,0,0,0,0,15.0
3,2023-06-16,210.0,0.125000,1.000000,2,0.0,1.0,0.0,0.175000,210.0,...,0,0,0,0,0,0,0,0,0,18.0
4,2023-06-16,270.0,0.166667,0.333333,3,0.0,0.0,1.0,0.138889,270.0,...,0,0,0,0,0,0,0,0,0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,525.0,0.250000,0.666667,2,0.0,1.0,0.0,0.297222,525.0,...,0,0,0,0,0,0,0,0,0,52.0
250,2023-07-07,630.0,0.125000,0.333333,3,0.0,0.0,1.0,0.088889,630.0,...,0,0,0,0,0,0,0,0,0,62.0
251,2023-07-07,690.0,0.250000,0.333333,0,0.0,0.0,0.0,0.313889,690.0,...,0,0,0,0,0,0,0,0,0,65.0
252,2023-07-07,795.0,0.250000,0.666667,3,0.0,0.0,1.0,0.227778,795.0,...,0,0,0,0,0,0,0,0,0,78.0


In [13]:
def drop_some_values(x, prob=0.95 ):
    if np.random.uniform() > prob:
        return 0
    return x

df['Duration'] = df['Duration'].apply(lambda x: drop_some_values(x))

df['Start Time'] = df['Start Time'] / max_time
df['True Start Time'] = df['Start Time']
df['Start Time'] = df['Start Time'].apply(lambda x: drop_some_values(x))
df.describe()

,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time,TS0,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
count,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000,...,254.0,254.0,254.0,254.0,254.0,254.0,254.0,254.0,254.0,254.000000
mean,0.277218,0.176345,0.417323,1.779528,0.110236,0.220472,0.409449,0.194641,0.296384,0.913386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.767717
std,0.200578,0.065922,0.299226,1.231429,0.313802,0.415384,0.492703,0.084242,0.193533,0.281824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.490903
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030556,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,0.093815,0.125000,0.333333,0.000000,0.000000,0.000000,0.000000,0.130556,0.125087,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.000000
50%,0.260598,0.166667,0.333333,2.000000,0.000000,0.000000,0.000000,0.175000,0.291869,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.000000
75%,0.458652,0.250000,0.666667,3.000000,0.000000,0.000000,1.000000,0.299306,0.458652,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.000000
max,0.625434,0.250000,1.000000,3.000000,1.000000,1.000000,1.000000,0.316667,0.625434,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.000000


In [14]:
df

,Date,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
0,2023-06-16,0.000000,0.166667,0.333333,3,0.0,0.0,1.0,0.138889,0.000000,...,0,0,0,0,0,0,0,0,0,0.0
1,2023-06-16,0.052120,0.125000,0.666667,0,0.0,0.0,0.0,0.186111,0.052120,...,0,0,0,0,0,0,0,0,0,6.0
2,2023-06-16,0.093815,0.166667,0.000000,1,1.0,0.0,0.0,0.113889,0.093815,...,0,0,0,0,0,0,0,0,0,15.0
3,2023-06-16,0.145935,0.125000,1.000000,2,0.0,1.0,0.0,0.175000,0.145935,...,0,0,0,0,0,0,0,0,0,18.0
4,2023-06-16,0.000000,0.166667,0.333333,3,0.0,0.0,1.0,0.138889,0.187630,...,0,0,0,0,0,0,0,0,0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2023-07-07,0.364837,0.250000,0.666667,2,0.0,1.0,0.0,0.297222,0.364837,...,0,0,0,0,0,0,0,0,0,52.0
250,2023-07-07,0.437804,0.125000,0.333333,3,0.0,0.0,1.0,0.088889,0.437804,...,0,0,0,0,0,0,0,0,0,62.0
251,2023-07-07,0.479500,0.250000,0.333333,0,0.0,0.0,0.0,0.313889,0.479500,...,0,0,0,0,0,0,0,0,0,65.0
252,2023-07-07,0.552467,0.250000,0.666667,3,0.0,0.0,1.0,0.227778,0.552467,...,0,0,0,0,0,0,0,0,0,78.0


### Test train dataset

In [15]:
X = df.drop(columns=['Date', 'True Duration', 'True Start Time', 'Label Number', 'Summ'])
y = df[['True Start Time', 'True Duration']]

In [16]:
len(X.columns)

126

In [17]:
y.describe()

,True Start Time,True Duration
count,254.000000,254.000000
mean,0.296384,0.194641
std,0.193533,0.084242
min,0.000000,0.030556
25%,0.125087,0.130556
50%,0.291869,0.175000
75%,0.458652,0.299306
max,0.625434,0.316667


In [18]:
X = X.to_numpy()
y = y.to_numpy()


In [19]:
y

array([[0.        , 0.13888889],
       [0.05211953, 0.18611111],
       [0.09381515, 0.11388889],
       [0.14593468, 0.175     ],
       [0.1876303 , 0.13888889],
       [0.25017373, 0.21388889],
       [0.30229326, 0.13055556],
       [0.33356498, 0.30555556],
       [0.41695622, 0.1       ],
       [0.45865184, 0.19166667],
       [0.50034746, 0.13055556],
       [0.55246699, 0.04166667],
       [0.58373871, 0.12777778],
       [0.        , 0.13888889],
       [0.05211953, 0.16944444],
       [0.09381515, 0.30277778],
       [0.16678249, 0.10833333],
       [0.20847811, 0.22222222],
       [0.26059764, 0.29722222],
       [0.33356498, 0.30555556],
       [0.41695622, 0.10833333],
       [0.45865184, 0.30555556],
       [0.53161918, 0.225     ],
       [0.60458652, 0.15      ],
       [0.        , 0.14166667],
       [0.05211953, 0.31388889],
       [0.12508687, 0.13055556],
       [0.17720639, 0.09722222],
       [0.21890202, 0.30555556],
       [0.29186935, 0.30833333],
       [0.

In [20]:
len(torch.tensor(y,dtype=torch.float32))

254

In [21]:
df[208:]

,Date,Start Time,Duration,Importance,Label Number,Label Number_1,Label Number_2,Label Number_3,True Duration,True Start Time,...,TS111,TS112,TS113,TS114,TS115,TS116,TS117,TS118,TS119,Summ
208,2023-07-04,0.000000,0.166667,0.333333,3,0.0,0.0,1.0,0.127778,0.000000,...,0,0,0,0,0,0,0,0,0,0.0
209,2023-07-04,0.052120,0.125000,0.666667,0,0.0,0.0,0.0,0.172222,0.052120,...,0,0,0,0,0,0,0,0,0,6.0
210,2023-07-04,0.093815,0.166667,0.000000,1,1.0,0.0,0.0,0.116667,0.093815,...,0,0,0,0,0,0,0,0,0,14.0
211,2023-07-04,0.145935,0.125000,1.000000,2,0.0,1.0,0.0,0.177778,0.145935,...,0,0,0,0,0,0,0,0,0,18.0
212,2023-07-04,0.187630,0.166667,0.333333,3,0.0,0.0,1.0,0.138889,0.187630,...,0,0,0,0,0,0,0,0,0,26.0
213,2023-07-04,0.250174,0.166667,0.666667,2,0.0,1.0,0.0,0.225000,0.250174,...,0,0,0,0,0,0,0,0,0,30.0
214,2023-07-04,0.302293,0.083333,0.000000,0,0.0,0.0,0.0,0.136111,0.302293,...,0,0,0,0,0,0,0,0,0,40.0
215,2023-07-04,0.333565,0.250000,0.333333,2,0.0,1.0,0.0,0.302778,0.333565,...,0,0,0,0,0,0,0,0,0,44.0
216,2023-07-04,0.416956,0.125000,0.333333,3,0.0,0.0,1.0,0.097222,0.416956,...,0,0,0,0,0,0,0,0,0,54.0
217,2023-07-04,0.458652,0.125000,0.666667,0,0.0,0.0,0.0,0.175000,0.458652,...,0,0,0,0,0,0,0,0,0,59.0


In [22]:
from sklearn.model_selection import train_test_split

class TaskDataset(Dataset):
  def __init__(self,X, y):
    self.x_train=torch.tensor(X,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.float32)

  def __len__(self):
    return len(self.y_train)

  def __getitem__(self,idx):
    return self.x_train[idx], self.y_train[idx]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state=42, shuffle=False)
datasets = {
    'train': TaskDataset(X_train, y_train),
    'test': TaskDataset(X_test, y_test)
}

batch_size = 1
dataloaders = {
    'train': DataLoader(datasets['train'], batch_size=batch_size, shuffle=False),
    # 'val': DataLoader(valData, batch_size=batch_size, shuffle=False),
    'test': DataLoader(datasets['test'], batch_size=batch_size, shuffle=False)
}

### Model designing

In [23]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [24]:
# Definition of the very simple network with 1 hidden layer
class SimpleScheduler(nn.Module):
    def __init__(self, input_n=126):
        # Invoke the parent's constructor
        super(SimpleScheduler, self).__init__()
        # nn implements feedworward layer as nn.Linear(a, b),
        # where a, b - input and output dims of weight matrix.
        # Bias is included by default.
        self.hidden = nn.Linear(input_n, 50)
        self.output = nn.Linear(50, 2)


    def forward(self, x):
        # You can write the result of sigmoid to a local variable,
        # but the common style in torch is to re-write x after computation of each layer
        x = F.relu(self.hidden(x))
        x = self.output(x)
        return x


model1 = SimpleScheduler().to(device)

print(f"SimpleScheduler model architecture:\n {model1}")
print("\n")

SimpleScheduler model architecture:
 SimpleScheduler(
  (hidden): Linear(in_features=126, out_features=50, bias=True)
  (output): Linear(in_features=50, out_features=2, bias=True)
)




In [25]:
# Set hyperparameters
# Number of iterations over the whole data set
epochs = 50
# Learning rate for Stochastic Gradient Descent
lr = 0.005
# SGD parameter to accelerate the optimization, check https://paperswithcode.com/method/sgd-with-momentum
momentum = 0.5
# Loss function - cross entropy, the multiclass variant of that you used in Lab 3
criterion = nn.HuberLoss()

In [26]:
from tqdm import tqdm

def train( model, device, train_loader, optimizer, epoch):
    # Do not forget to set train() to update weights of model
    model.train()
    # A wrapper over data loader to show progress bar
    bar = tqdm(train_loader)
    iteration = 0
    overall_loss = 0
    for data, target in bar:
        # Device of data and model must be the same
        data, target = data.to(device), target.to(device)
        # To avoid an accumulation of gradients
        optimizer.zero_grad()
        # Prediction
        output = model(data)
        # Error between prediction and ground truth
        loss = criterion(output, target)
        # Compute gradient
        loss.backward()
        # Update params of model
        optimizer.step()

        iteration += 1
        overall_loss += loss.item()
        bar.set_postfix({"Loss": format(overall_loss/iteration, '.6f')})

def test( model, device, test_loader):
    # Do not forget to set eval() to keep model's params the same
    model.eval()
    test_loss = 0
    correct = 0
    mse_loss = nn.MSELoss()
    mse_loss_val = 0
    ind = 0
    with torch.no_grad():
        for data, target in test_loader:
            # print(f"target shape is {target.shape}")
            data, target = data.to(device), target.to(device)
            output = model(data)
            # print(f"output shape is {output.shape}")
            # Sum up batch loss
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            mse_loss_val += mse_loss(output, target).item()
            # mse_loss += sum()
            # pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # print(output.shape)
            # targ = target.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # print(target, "\n", output)

            # correct += pred.eq(targ.view_as(pred)).sum().item()

    # test_loss /= len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss} mse loss: {mse_loss_val}")

In [27]:
# Train the model
optimizer = optim.Adam(model1.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model1, device, dataloaders['train'], optimizer, epoch)
    test(model1, device, dataloaders['test'])

# Optionally, you can save the params of model to file
#torch.save(model.state_dict(), "mnist_model.pt")


100%|██████████| 208/208 [00:01<00:00, 192.84it/s, Loss=0.009354]


tensor([[0.0000, 0.1278]]) 
 tensor([[0.0044, 0.1361]])
tensor([[0.0521, 0.1722]]) 
 tensor([[0.0355, 0.2439]])
tensor([[0.0938, 0.1167]]) 
 tensor([[0.0959, 0.1439]])
tensor([[0.1459, 0.1778]]) 
 tensor([[0.1141, 0.2170]])
tensor([[0.1876, 0.1389]]) 
 tensor([[0.1825, 0.1625]])
tensor([[0.2502, 0.2250]]) 
 tensor([[0.1890, 0.2394]])
tensor([[0.3023, 0.1361]]) 
 tensor([[0.3086, 0.2075]])
tensor([[0.3336, 0.3028]]) 
 tensor([[0.3159, 0.2611]])
tensor([[0.4170, 0.0972]]) 
 tensor([[0.4099, 0.1824]])
tensor([[0.4587, 0.1750]]) 
 tensor([[0.4437, 0.1820]])
tensor([[0.5003, 0.1361]]) 
 tensor([[0.5286, 0.1494]])
tensor([[0.5525, 0.0500]]) 
 tensor([[0.5612, 0.1267]])
tensor([[0.5837, 0.1444]]) 
 tensor([[0.5535, 0.1064]])
tensor([[0.0000, 0.1306]]) 
 tensor([[0.0044, 0.1361]])
tensor([[0.0521, 0.3028]]) 
 tensor([[0.0343, 0.2479]])
tensor([[0.1251, 0.1250]]) 
 tensor([[0.1162, 0.1443]])
tensor([[0.1772, 0.1083]]) 
 tensor([[0.1499, 0.1301]])
tensor([[0.2189, 0.3000]]) 
 tensor([[0.1939, 0.

100%|██████████| 208/208 [00:00<00:00, 220.08it/s, Loss=0.001496]


tensor([[0.0000, 0.1278]]) 
 tensor([[0.0027, 0.1361]])
tensor([[0.0521, 0.1722]]) 
 tensor([[0.0311, 0.2388]])
tensor([[0.0938, 0.1167]]) 
 tensor([[0.0772, 0.1296]])
tensor([[0.1459, 0.1778]]) 
 tensor([[0.0924, 0.1999]])
tensor([[0.1876, 0.1389]]) 
 tensor([[0.1210, 0.1317]])
tensor([[0.2502, 0.2250]]) 
 tensor([[0.1448, 0.2178]])
tensor([[0.3023, 0.1361]]) 
 tensor([[0.2342, 0.2048]])
tensor([[0.3336, 0.3028]]) 
 tensor([[0.2453, 0.2720]])
tensor([[0.4170, 0.0972]]) 
 tensor([[0.3108, 0.1297]])
tensor([[0.4587, 0.1750]]) 
 tensor([[0.3499, 0.1623]])
tensor([[0.5003, 0.1361]]) 
 tensor([[0.4029, 0.1376]])
tensor([[0.5525, 0.0500]]) 
 tensor([[0.4322, 0.1144]])
tensor([[0.5837, 0.1444]]) 
 tensor([[0.4513, 0.1048]])
tensor([[0.0000, 0.1306]]) 
 tensor([[0.0027, 0.1361]])
tensor([[0.0521, 0.3028]]) 
 tensor([[0.0299, 0.2465]])
tensor([[0.1251, 0.1250]]) 
 tensor([[0.0895, 0.1270]])
tensor([[0.1772, 0.1083]]) 
 tensor([[0.1146, 0.1023]])
tensor([[0.2189, 0.3000]]) 
 tensor([[0.1729, 0.

100%|██████████| 208/208 [00:00<00:00, 229.82it/s, Loss=0.002345]


...

tensor([[0.0000, 0.1278]]) 
 tensor([[0.0110, 0.1302]])
tensor([[0.0521, 0.1722]]) 
 tensor([[0.1023, 0.2068]])
tensor([[0.0938, 0.1167]]) 
 tensor([[0.1480, 0.0928]])
tensor([[0.1459, 0.1778]]) 
 tensor([[0.2000, 0.1528]])
tensor([[0.1876, 0.1389]]) 
 tensor([[0.2341, 0.0997]])
tensor([[0.2502, 0.2250]]) 
 tensor([[0.2935, 0.2175]])
tensor([[0.3023, 0.1361]]) 
 tensor([[0.3511, 0.1090]])
tensor([[0.3336, 0.3028]]) 
 tensor([[0.3905, 0.2727]])
tensor([[0.4170, 0.0972]]) 
 tensor([[0.4042, 0.1133]])
tensor([[0.4587, 0.1750]]) 
 tensor([[0.4565, 0.2161]])
tensor([[0.5003, 0.1361]]) 
 tensor([[0.5118, 0.1219]])
tensor([[0.5525, 0.0500]]) 
 tensor([[0.5627, 0.1259]])
tensor([[0.5837, 0.1444]]) 
 tensor([[0.5921, 0.1282]])
tensor([[0.0000, 0.1306]]) 
 tensor([[0.0110, 0.1302]])
tensor([[0.0521, 0.3028]]) 
 tensor([[0.0872, 0.2445]])
tensor([[0.1251, 0.1250]]) 
 tensor([[0.1834, 0.0957]])
tensor([[0.1772, 0.1083]]) 
 tensor([[0.2303, 0.0994]])
tensor([[0.2189, 0.3000]]) 
 tensor([[0.2840, 0.

100%|██████████| 208/208 [00:02<00:00, 89.92it/s, Loss=0.001388]


tensor([[0.0000, 0.1278]]) 
 tensor([[0.0346, 0.1319]])
tensor([[0.0521, 0.1722]]) 
 tensor([[0.1261, 0.2027]])
tensor([[0.0938, 0.1167]]) 
 tensor([[0.1725, 0.1023]])
tensor([[0.1459, 0.1778]]) 
 tensor([[0.2037, 0.1629]])
tensor([[0.1876, 0.1389]]) 
 tensor([[0.2533, 0.1056]])
tensor([[0.2502, 0.2250]]) 
 tensor([[0.3156, 0.2188]])
tensor([[0.3023, 0.1361]]) 
 tensor([[0.3781, 0.1108]])
tensor([[0.3336, 0.3028]]) 
 tensor([[0.4085, 0.2702]])
tensor([[0.4170, 0.0972]]) 
 tensor([[0.4116, 0.1122]])
tensor([[0.4587, 0.1750]]) 
 tensor([[0.4479, 0.2034]])
tensor([[0.5003, 0.1361]]) 
 tensor([[0.5031, 0.1183]])
tensor([[0.5525, 0.0500]]) 
 tensor([[0.5584, 0.1220]])
tensor([[0.5837, 0.1444]]) 
 tensor([[0.5984, 0.1247]])
tensor([[0.0000, 0.1306]]) 
 tensor([[0.0346, 0.1319]])
tensor([[0.0521, 0.3028]]) 
 tensor([[0.1117, 0.2445]])
tensor([[0.1251, 0.1250]]) 
 tensor([[0.2043, 0.1036]])
tensor([[0.1772, 0.1083]]) 
 tensor([[0.2449, 0.1053]])
tensor([[0.2189, 0.3000]]) 
 tensor([[0.3029, 0.

100%|██████████| 208/208 [00:01<00:00, 104.86it/s, Loss=0.000853]


tensor([[0.0000, 0.1278]]) 
 tensor([[0.0035, 0.1335]])
tensor([[0.0521, 0.1722]]) 
 tensor([[0.0992, 0.2148]])
tensor([[0.0938, 0.1167]]) 
 tensor([[0.1347, 0.0994]])
tensor([[0.1459, 0.1778]]) 
 tensor([[0.1888, 0.1646]])
tensor([[0.1876, 0.1389]]) 
 tensor([[0.2250, 0.1009]])
tensor([[0.2502, 0.2250]]) 
 tensor([[0.2892, 0.2225]])
tensor([[0.3023, 0.1361]]) 
 tensor([[0.3460, 0.1107]])
tensor([[0.3336, 0.3028]]) 
 tensor([[0.3835, 0.2702]])
tensor([[0.4170, 0.0972]]) 
 tensor([[0.4310, 0.1041]])
tensor([[0.4587, 0.1750]]) 
 tensor([[0.4432, 0.2157]])
tensor([[0.5003, 0.1361]]) 
 tensor([[0.4897, 0.1135]])
tensor([[0.5525, 0.0500]]) 
 tensor([[0.5383, 0.1213]])
tensor([[0.5837, 0.1444]]) 
 tensor([[0.5733, 0.1460]])
tensor([[0.0000, 0.1306]]) 
 tensor([[0.0035, 0.1335]])
tensor([[0.0521, 0.3028]]) 
 tensor([[0.0838, 0.2521]])
tensor([[0.1251, 0.1250]]) 
 tensor([[0.1681, 0.1000]])
tensor([[0.1772, 0.1083]]) 
 tensor([[0.2152, 0.1007]])
tensor([[0.2189, 0.3000]]) 
 tensor([[0.2735, 0.

In [28]:
def float_to_schedule(input):
    start, duration = input[0], input[1]
    start *= max_time
    hours = start // 24
    minutes = start % 24
    duration *= DUR
    return hours, minutes, duration


In [29]:
df_out = df_out.reset_index(drop=True)
df_out

,Date,Start Time,Duration,Importance,Label Number,Label Name,True Duration,True Hours,True Minutes
0,2023-07-04,07:00,60,1,3,Daily Routine,46,0.0,0.0
1,2023-07-04,08:15,45,2,0,Sport,62,3.0,3.0
2,2023-07-04,09:15,60,0,1,Hobby Passive,42,5.0,15.0
3,2023-07-04,10:30,45,3,2,Work&Study,64,8.0,18.0
4,2023-07-04,11:30,60,1,3,Daily Routine,50,11.0,6.0
5,2023-07-04,13:00,60,2,2,Work&Study,81,15.0,0.0
6,2023-07-04,14:15,30,0,0,Sport,49,18.0,3.0
7,2023-07-04,15:00,90,1,2,Work&Study,109,20.0,0.0
8,2023-07-04,17:00,45,1,3,Daily Routine,35,25.0,0.0
9,2023-07-04,18:00,45,2,0,Sport,63,27.0,12.0


In [30]:
# out_df = df[-1*y_test.shape[0]:]
with torch.no_grad():
    ind = 0
    for data, target in dataloaders['test']:
        # print(f"target shape is {target.shape}")
        data, target = data.to(device), target.to(device)
        output = model1(data).cpu().numpy()
        for i in output:
            df_out.loc[ind, ['Pred Hours', 'Pred Min', 'Pred Dur']] = float_to_schedule(i)
            ind+=1
df_out

,Date,Start Time,Duration,Importance,Label Number,Label Name,True Duration,True Hours,True Minutes,Pred Hours,Pred Min,Pred Dur
0,2023-07-04,07:00,60,1,3,Daily Routine,46,0.0,0.0,0.0,5.024813,48.068474
1,2023-07-04,08:15,45,2,0,Sport,62,3.0,3.0,5.0,22.780893,77.327356
2,2023-07-04,09:15,60,0,1,Hobby Passive,42,5.0,15.0,8.0,1.786469,35.796914
3,2023-07-04,10:30,45,3,2,Work&Study,64,8.0,18.0,11.0,7.732807,59.247749
4,2023-07-04,11:30,60,1,3,Daily Routine,50,11.0,6.0,13.0,11.725180,36.308294
5,2023-07-04,13:00,60,2,2,Work&Study,81,15.0,0.0,17.0,8.123688,80.102010
6,2023-07-04,14:15,30,0,0,Sport,49,18.0,3.0,20.0,17.963755,39.837222
7,2023-07-04,15:00,90,1,2,Work&Study,109,20.0,0.0,22.0,23.808654,97.285523
8,2023-07-04,17:00,45,1,3,Daily Routine,35,25.0,0.0,25.0,20.169923,37.474966
9,2023-07-04,18:00,45,2,0,Sport,63,27.0,12.0,26.0,13.793479,77.653738
